### fmri nonlinear

In [1]:
import torch
from dagma import utils
from dagma.linear import DagmaLinear
from dagma.nonlinear import DagmaMLP, DagmaNonlinear
import numpy as np
from auxiliary_funcs import *

In [2]:
X = np.concatenate([[np.eye(164)]] * 67, axis=0)
X = np.transpose(X , (1,2,0))

In [3]:
data_normalized=np.load( 'connectome_67subs_invnodevol .npy') 
s_connectome_probb=np.zeros((164,164,67))
s_connectome_probb1=np.zeros((164,164,67))

for i in range(67):
    for j in range (164): 
        s_connectome_probb[j,:,i]=(data_normalized[j,:,i]/(data_normalized[j,:,i].sum()))
        s_connectome_probb1[j,:,i]=s_connectome_probb[j,:,i]
       # s_connectome_probb[s_connectome_probb == -inf] = -5
    MAX=max(s_connectome_probb[:,:,i].flatten())
    MIN=min(s_connectome_probb[:,:,i].flatten())
    s_connectome_probb[:,:,i]=(s_connectome_probb[:,:,i]-MIN)/(MAX-MIN)
        
# s_connectome_probb[s_connectome_probb1 == -inf] = 0
# plt.hist(.5*s_connectome_probb[:,:,1].flatten(),bins=10 ,range=(0,0.1) )

In [4]:
s_connectome_probb2 = s_connectome_probb + X
np.save('DTI_conectome' ,s_connectome_probb)

In [68]:
edges = int(0.85 * 164*164)

In [69]:
sc = directed_mrtx(np.expand_dims(s_connectome_probb ,3) , edges)

In [70]:
bsc = np.sum(sc , 2)[:,:,0] 
bsc = (bsc > 37)*1
bsc.sum()/(164*164) , bsc.shape

(0.8577111838191552, (164, 164))

In [71]:
np.save('SC_binary' ,bsc)

In [80]:
#dataset and loading and saving adds 
data_folder = 'Data/'  
results_folder = 'outcomes/'
data_name = 'mean1_dataset'
prior_name = '_DTI_dataset'
# prior_name = ''
# data_name = 'mean2_dataset'
model_type = 'ldagma'
ul = '_'
dataset = np.load(data_folder + data_name + '.npy')
print("********"," dataset name is " , data_name , '  ' , model_type  , " ********")
#prepare the hyperparameters list
# sparsity_coef_list = [0.008]
sparsity_coef_list = [0.08]
th_coef_list = [0.001]
sub_nums = 62
node_nums = 164
num_samples = 1200
#model and training
for th_id , th_coef in enumerate(th_coef_list) :
    print(f"th id : {th_id}")
    for sp_id , sp_coef in enumerate(sparsity_coef_list):
        print(f"sp_id : {sp_id}")
        W_est = np.zeros((node_nums , node_nums, sub_nums))
        save_name = [data_name + prior_name + ul + model_type+ ul + 'num_subs_' + 
                     str(sub_nums) + ul + 'sparsity_' + str(sp_coef) +
                     ul + 'thershold' + ul + str(th_coef) ][0]

        add_strc = data_folder + "SC_binary.npy"
        
    B_est = pfdr_(results_folder+ save_name , add_strc ,num_ed = 5 ,num_div = 5, th =25)
        # connectivity_map(add1 = add_strc , add2 = results_folder+ save_name ,
        #                     edges =2000, name1 = 'Structural' , name2 = 'Estimated' , th=25)

********  dataset name is  mean1_dataset    ldagma  ********
th id : 0
sp_id : 0
estimated matrix edges are  :  850564
estimated matrix edges are  :  1651100
num_div is :  6
expected num edges is :  2778
real num edges is :  190
num of diff_est_strc   36
pfdr percentage is : 18.947368421052634
**********
num_div is :  7
expected num edges is :  2381
real num edges is :  118
num of diff_est_strc   20
pfdr percentage is : 16.94915254237288
**********
num_div is :  8
expected num edges is :  2083
real num edges is :  66
num of diff_est_strc   10
pfdr percentage is : 15.151515151515152
**********
num_div is :  9
expected num edges is :  1852
real num edges is :  48
num of diff_est_strc   7
pfdr percentage is : 14.583333333333334
**********
num_div is :  10
expected num edges is :  1667
real num edges is :  32
num of diff_est_strc   3
pfdr percentage is : 9.375
**********


In [81]:
add2 = results_folder+ save_name + '.npy'
res1 = np.expand_dims(np.load(add2),3)
bs1 = extract_conn(res1)

In [82]:
edges_nums = [100 , 200 , 300 , 400 ] 
for edges_num in edges_nums :
    bs2 = directed_mrtx(bs1 , edges_num )
    bs2[:,:,0].sum()
    bs2 = np.sum(bs2[:,:,:,0] , 2)
    bs3 = (bs2 >9)*1
    # print("bs3 : " ,bs3.sum())
    print(((bs3 - bsc)>0).sum()/bs3.sum())

/tmp/ipykernel_1193273/3795451858.py:8: RuntimeWarning: invalid value encountered in scalar divide
  print(((bs3 - bsc)>0).sum()/bs3.sum())


nan
0.0
0.03333333333333333
0.028985507246376812


In [76]:
edges_nums = [100 , 200 , 300 , 400 , 500 , 600 , 1800] 
for edges_num in edges_nums :
    bs2 = directed_mrtx(bs1 , edges_num )
    bs2[:,:,0].sum()
    bs2 = np.sum(bs2[:,:,:,0] , 2)
    bs3 = (bs2 > 40)*1
    # print("bs3 : " ,bs3.sum())
    print(((bs3 - bsc)>0).sum()/bs3.sum())

/tmp/ipykernel_1193273/4104769046.py:8: RuntimeWarning: invalid value encountered in scalar divide
  print(((bs3 - bsc)>0).sum()/bs3.sum())


nan
nan
nan
nan
nan
nan
nan
